## Analysis of used cars from eBay Kleinanzeigen
This is a German eBay website. The dataset was originally scraped and uploaded to [Kaggle](https://www.kaggle.com/orgesleka/used-cars-database/data). We've made a few modifications from the original dataset that was uploaded to Kaggle.

- We sampled 50,000 data points from the full dataset, to ensure your code runs quickly in our hosted environment
- We dirtied the dataset a bit to more closely resemble what you would expect from a scraped dataset (the version uploaded to Kaggle was cleaned to be easier to work with)

The aim of this project is to clean the data and analyze the included used car listings. You'll also become familiar with some of the unique benefits jupyter notebook provides for pandas.

In [1]:
import pandas as pd
import numpy as np

In [2]:
# if read in file without assinging encoding
# it will default to "UTF-8"
# try "Latin-1" or "Windows-1252" until you're able to read the file
autos = pd.read_csv("autos.csv", encoding = "Windows-1252")

In [3]:
# review the first 5 rows 
autos.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


In [4]:
# check the information of data
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

### Cleaning Column Names

We can make the following observations:
* The dataset contains 20 columns, most of which are strings.
* Some columns have null values, but none have more than ~20% null values.
* The column name use camelcase instead of Python's preferred snakecase, which means we can't just replace spaces with underscore.

In [5]:
# print an array of the existing column names
autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

In [6]:
# rename below columns with funtion "clean_column"
def cleaning(col):
    """
    column names below use camelcase instead of Python's preferred
    snakecase, so we need to modify column names manually
    """
    col = col.replace("yearOfRegistration", "registration_year")
    col = col.replace("monthOfRegistration", "registration_month")
    col = col.replace("notRepairedDamage", "unrepaired_damage")
    col = col.replace("dateCreated", "ad_created")
    col = col.lower()
    return col



# use an empty list and iterate column names to 
# rename with funtion "cleaning"
cleaned_column = []
for i in autos.columns:
    updated_name = cleaning(i)
    cleaned_column.append(updated_name)
    
# assign the modified column names back to DataFrame.columns
autos.columns = cleaned_column
print(autos.columns)


Index(['datecrawled', 'name', 'seller', 'offertype', 'price', 'abtest',
       'vehicletype', 'registration_year', 'gearbox', 'powerps', 'model',
       'odometer', 'registration_month', 'fueltype', 'brand',
       'unrepaired_damage', 'ad_created', 'nrofpictures', 'postalcode',
       'lastseen'],
      dtype='object')


### Initial Exploration and Cleaning
Now let's do some basic data exploration to determine what other cleaning tasks need to be done. Initially we will look for: - Text columns where all or almost all values are the same. These can often be dropped as they don't have useful information for analysis. - Examples of numeric data stored as text which can be cleaned and converted.

In [7]:
# look at descriptive statistics for all columns
autos.describe()

,registration_year,powerps,registration_month,nrofpictures,postalcode
count,50000.000000,50000.000000,50000.000000,50000.0,50000.000000
mean,2005.073280,116.355920,5.723360,0.0,50813.627300
std,105.712813,209.216627,3.711984,0.0,25779.747957
min,1000.000000,0.000000,0.000000,0.0,1067.000000
25%,1999.000000,70.000000,3.000000,0.0,30451.000000
50%,2003.000000,105.000000,6.000000,0.0,49577.000000
75%,2008.000000,150.000000,9.000000,0.0,71540.000000
max,9999.000000,17700.000000,12.000000,0.0,99998.000000


In [8]:
# check more
autos.head()

,datecrawled,name,seller,offertype,price,abtest,vehicletype,registration_year,gearbox,powerps,model,odometer,registration_month,fueltype,brand,unrepaired_damage,ad_created,nrofpictures,postalcode,lastseen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


In [9]:
# Drop the columns that mostly values are the same and 
# not critical for further analysis
autos = autos.drop(["nrofpictures","seller", "offertype"], axis = 1)

In [10]:
# price and odometer columns are numeric values stored as text

print(autos["price"].describe())
print('--------------------')
print(autos["odometer"].describe())

count     50000
unique     2357
top          $0
freq       1421
Name: price, dtype: object
--------------------
count         50000
unique           13
top       150,000km
freq          32424
Name: odometer, dtype: object


In [11]:
# remove any non-numeric characters
# convert the column to numeric dtype
autos["price"] = autos["price"].str.replace("$","").str.replace(",","").astype(int)
autos["odometer"] = autos["odometer"].str.replace("km","").str.replace(",","").astype(int)

# rename the column
autos.rename({"odometer":"odometer_km"}, axis = 1, inplace = True)

In [12]:
# see how many unique values of price
autos["price"].unique().shape[0]

2357

In [13]:
# Price suddenly arises from $350,000 to $999,990
# Given we are studying used cars which should be lower expenses
# we are more likely to drop the outliers of data
autos["price"].sort_values(ascending = True).tail(20)

47337      259000
12682      265000
35923      295000
34723      299000
14715      345000
36818      350000
37585      999990
514        999999
43049      999999
22947     1234566
7814      1300000
47634     3890000
11137    10000000
24384    11111111
2897     11111111
27371    12345678
47598    12345678
39377    12345678
42221    27322222
39705    99999999
Name: price, dtype: int64

In [14]:
# keep the data which price lies within $350,000
autos = autos[autos["price"].between(1,350001)]

In [15]:
# no obvious data need to be removed
autos["odometer_km"].value_counts(ascending = False)

150000    31414
125000     5057
100000     2115
90000      1734
80000      1415
70000      1217
60000      1155
50000      1012
5000        836
40000       815
30000       780
20000       762
10000       253
Name: odometer_km, dtype: int64

In [16]:
# 1,435 outlier data was removed
autos["price"].describe()

count     48565.000000
mean       5888.935591
std        9059.854754
min           1.000000
25%        1200.000000
50%        3000.000000
75%        7490.000000
max      350000.000000
Name: price, dtype: float64

### Exploring the date columns

In [17]:
# use str[:10] to extract "yyyy-mm-dd"
# "series.value_counts()" to see distribution
# "normalize = True" to see percentage
# chain "series.sort_values(ascending = True)" to see earliest to latest

# date_crawled
    
date_crawled = autos["datecrawled"].str[:10]
date_crawled = date_crawled.value_counts(normalize = True, dropna = False).sort_index(ascending = True)
print(date_crawled.head())


2016-03-05    0.025327
2016-03-06    0.014043
2016-03-07    0.036014
2016-03-08    0.033296
2016-03-09    0.033090
Name: datecrawled, dtype: float64


In [18]:
# ad_created

ad_created = autos["ad_created"].str[:10]
ad_created = ad_created.value_counts(normalize = True, dropna= False).sort_values(ascending = True)
print(ad_created.head())


2016-01-13    0.000021
2015-06-11    0.000021
2016-02-07    0.000021
2016-02-11    0.000021
2016-01-22    0.000021
Name: ad_created, dtype: float64


In [19]:
# last_seen

last_seen = autos["lastseen"].str[:10]
last_seen = last_seen.value_counts(normalize = True, dropna = False).sort_values(ascending = True)
print(last_seen.head())


2016-03-05    0.001071
2016-03-06    0.004324
2016-03-07    0.005395
2016-03-18    0.007351
2016-03-08    0.007413
Name: lastseen, dtype: float64


In [20]:
# use Series.describe() to understand the distribution
# of registration_year

autos["registration_year"].describe()

count    48565.000000
mean      2004.755421
std         88.643887
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

### Dealing with Incorrect Registration Year Data

Based on exploration above, 
* The min value is the year of 1000, before car is invented
* The max value is the year of 9999, many years into the future

According to our common sense, the car was invented in the early 1900s. Besides, the car can't be registered after this data was collected in 2016. 

We'd like to count the number with cars that fall outside the 1900 - 2016. 

Let's research a little bit further.

In [21]:
# count number of outlier
registration_bool_1 = (autos["registration_year"] > 2016)
print('Number of which registration year after 2016:',sum(registration_bool_1))
registration_bool_2 = (autos["registration_year"] < 1900)
print('Number of which registration year before 1900:',sum(registration_bool_2))



Number of which registration year after 2016: 1879
Number of which registration year before 1900: 5


In [22]:
# use boolean to keep value which 
# registration_year within the range 1900 and 2016
bool_1 = (autos["registration_year"] > 1900)
bool_2 = (autos["registration_year"] < 2017)
autos = autos.loc[bool_1 & bool_2]

# calculate the distribution of remaining values 
autos["registration_year"].value_counts(normalize = True, ascending = False).head(10)


2000    0.067608
2005    0.062895
1999    0.062060
2004    0.057904
2003    0.057818
2006    0.057197
2001    0.056468
2002    0.053255
1998    0.050620
2007    0.048778
Name: registration_year, dtype: float64

We found out most registration year was: 2000, 2005, 1999, 2004, 2003,...,etc, which means most used cars are over 10 years and if you purchase them, you might need to take good care to save repair expenses.

### Exploring Price by Brand

In [23]:
# identify the unique values we want to aggregate by
brands = autos["brand"].value_counts(normalize=True)
print(brands.head(10))

# select Volkswagen, BMW, Opel, Mercedes Benz, Audi and Ford 
# to analyze its mean price.

volkswagen       0.211264
bmw              0.110045
opel             0.107581
mercedes_benz    0.096463
audi             0.086566
ford             0.069900
renault          0.047150
peugeot          0.029841
fiat             0.025642
seat             0.018273
Name: brand, dtype: float64


Based on result above, we can tell German really loves German-brand as headquarter of top 5 brands on the list are located in German.
Volkswagen(21%), BMW(11%), Opel(10%), Mercedes Benz(9%) and Audi(8%) are 59% in total.

Except for German brands, no.6 on the list is American-brand Ford(6.9%). no.7 Renault(4.7%) and no.8 Peugeot(2.9%) are French-brand. 

In [24]:
# select the brands which is over 5% in the market
brands_over_5 = brands.index[:6]

In [25]:
# Try to calculate mean price of Volkswagen
# and then finish a for-loop

volkswagen = (autos["brand"] == "volkswagen")
volkswagen_price_mean = autos["price"][volkswagen].mean().round(2)
print(volkswagen_price_mean)

# try to use a for-loop to finish mean values of the rest

5402.41


In [26]:
# create an empty list to store mean prices
brand_mean_price = {}

# use a for-loop to collect mean price
for brand_name in brands_over_5:
    bool_1 = (autos["brand"] == brand_name)
    mean_price = autos["price"][bool_1].mean().round(2)
    brand_mean_price[brand_name] = mean_price

print(brand_mean_price)

{'opel': 2975.24, 'volkswagen': 5402.41, 'bmw': 8332.82, 'mercedes_benz': 8628.45, 'ford': 3749.47, 'audi': 9336.69}


### Storing Aggregation Data in a DataFrame

Among those brands with over 5% market share in eBay, we can classify brands by price as below.
* Very expensive(>8000): Audi,Mercedes Benz, BMW
* In between: Volkswagen
* Less expensive(<4000): Ford, Opel

Let's use aggregation to understand if there's any link between average mileage and average price.

In [27]:
#  calculate mean mileage of 6 brands
brand_mean_mileage = {}

for brand in brands_over_5:
    bool_2 = (autos["brand"] == brand)
    mean_mileage = autos["odometer_km"][bool_2].mean().round(2)
    brand_mean_mileage[brand] = mean_mileage
    
print(brand_mean_mileage)


{'opel': 129310.04, 'volkswagen': 128707.16, 'bmw': 132572.51, 'mercedes_benz': 130788.36, 'ford': 124266.01, 'audi': 129157.39}


In [28]:
# convert both dictionaries to series object
bmp_series = pd.Series(brand_mean_price)
bmm_series = pd.Series(brand_mean_mileage)

# create a dataframe from the first series object
# df = pd.DataFrame({"mean_price":bmp_series, "mean_mileage":bmm_series})
df = pd.DataFrame(bmp_series, columns=["mean_price"])

# assign the other series as a new column
df["mean_mileage"] = bmm_series

# investigate the relationship between price & mileage
print(df.sort_values("mean_price", ascending = False))
print('------------------------------------------')
print(df.corr())


               mean_price  mean_mileage
audi              9336.69     129157.39
mercedes_benz     8628.45     130788.36
bmw               8332.82     132572.51
volkswagen        5402.41     128707.16
ford              3749.47     124266.01
opel              2975.24     129310.04
------------------------------------------
              mean_price  mean_mileage
mean_price      1.000000      0.612364
mean_mileage    0.612364      1.000000


For very expensive brands(Audi,Mercedes Benz, BMW), the more mileage, the price is less expensive.

For less expensive brands(Ford, Opel), it's the same that the more mileage, cars would be sold at lower price.

### Identify categorical data that uses german words, translate them and map the values to their english counterparts

In [29]:
# Below columns have german words and we'd like to translate into English
print(autos["vehicletype"].unique())
print()
print(autos["gearbox"].unique())
print()
print(autos["fueltype"].unique())
print()
print(autos["unrepaired_damage"].unique())

['bus' 'limousine' 'kleinwagen' 'kombi' nan 'coupe' 'suv' 'cabrio'
 'andere']

['manuell' 'automatik' nan]

['lpg' 'benzin' 'diesel' nan 'cng' 'hybrid' 'elektro' 'andere']

['nein' nan 'ja']


In [30]:
# build a vehicle type dictionary
vehicletype_dict = {"bus":"bus", "limousine":"limousine",
                    "coupe":"coupe", "suv":"suv",
                    "kleinwagen":"small car", "kombi":"combi",
                   "cabrio":"convertible", "andere":"other"}

# mapping dictionary to column
autos["vehicletype"] = autos["vehicletype"].map(vehicletype_dict)

print(autos["vehicletype"].unique())

['bus' 'limousine' 'small car' 'combi' nan 'coupe' 'suv' 'convertible'
 'other']


In [31]:
# grearbox dictionary
gearbox_dict = {"manuell":"manually", "automatik":"automatic"}

# fuel type dictionary
fueltype_dict = {"lpg":"lpg", "benzin":"petrol", "diesel":"diesel",
                "cng":"cng", "hybrid":"hybrid", "elektro":"electro",
                "andere":"other"}
# unrepaired damage
ud_dict = {"nein":"no", "ja":"yes"}


# mapping gearbox
autos["gearbox"] = autos["gearbox"].map(gearbox_dict)
print(autos["gearbox"].unique())
# mapping fueltype
autos["fueltype"] = autos["fueltype"].map(fueltype_dict)
print(autos["fueltype"].unique())
# mapping unrepaired damage
autos["unrepaired_damage"] = autos["unrepaired_damage"].map(ud_dict)
print(autos["unrepaired_damage"].unique())

['manually' 'automatic' nan]
['lpg' 'petrol' 'diesel' nan 'cng' 'hybrid' 'electro' 'other']
['no' nan 'yes']


### Convert the dates to be uniform numeric data, so `"2016-03-21"` becomes the integer `20160321`

In [32]:
# extract datecrawled strings
date = autos["datecrawled"].str[:10]
# remove "-" like "20160321" and return to column "datecrawled"
autos["datecrawled"] = date.str.replace("-","")
autos["datecrawled"].head()

0    20160326
1    20160404
2    20160326
3    20160312
4    20160401
Name: datecrawled, dtype: object

### Find the most common brand/model combinations

In [33]:
# Find the most common brand/model combinations
model = autos["model"].value_counts(normalize=True)
print(model.head(5))
# Top 5 model on the eBay is: Gold, Andere, 3er, polo and corsa

def find_brand(model_name):
    bool_1 = (autos["model"]==model_name)
    print(autos[bool_1]["brand"])
    

golf      0.083326
andere    0.075818
3er       0.058780
polo      0.036167
corsa     0.035785
Name: model, dtype: float64


#### Top 5 brand/model on the eBay is:
1. Volkswagen/Golf (8.3%)
2. Peugeot/Andere (7.5%)
3. BMW/3er (5.8%)
4. Volkswagen/Polo (3.6%)
5. Opel/Corsa (3.5%)

### Split the `odometer_km` into groups, and use aggregation to see if average prices follows any patterns based on the milage.

In [34]:
# spilt the odometer_km into groups
# use aggregation to see if average prices
# follows any patterns based on the milage

# find out frequency of milage and build a list for later use
milage_percentage = autos["odometer_km"].value_counts(normalize=True)
milage_list = milage_percentage.index[:]


# build an empty dictionary to find average price by milage
milage_price = {}

# use a for-loop and boolean method to calculate average price
for milage in milage_list:
    bool_3 = (autos["odometer_km"] == milage)
    A = autos["price"][bool_3].mean().round()
    milage_price[milage] = A
    
# print(milage_price) to check

# establish a dataframe for analysis
mp_series = pd.Series(milage_price)
mp = pd.DataFrame(mp_series, columns = ["mean price"])
print(mp)

        mean price
5000        8874.0
10000      20551.0
20000      18448.0
30000      16609.0
40000      15500.0
50000      13812.0
60000      12385.0
70000      10927.0
80000       9722.0
90000       8465.0
100000      8133.0
125000      6214.0
150000      3768.0


Based on dataframe above, we can seperate data into 2 groups.

For milage less than 5000km, we can see the average price is also lower. We have to go deeper to analyze if there's another variable to affect price, brand and model for example.

For another group which milage over 5000km to 150000km, we can see the more milage car is, the more depreciation it is which means the car is sold at cheaper price.

I'm not sure how much is new car sold in Germany. In my country, most of new cars was sold at about $30000~$40000 tariff included. If you'd like to spend less money and have better quality, you could try to find cars which milage is in about 20000km.

### How much cheaper are cars with damage than their non-damaged counterparts?

In [47]:
# How much cheaper are cars with damage
# than their non-damage counterparts
# create a percentage 
damage_or_not = autos["unrepaired_damage"].value_counts(normalize=True)
yes_or_no = damage_or_not.index[:]

# create an empty dictionary
damage_price = {}

# use a for-loop and boolean method to calculate average price
for row in yes_or_no:
    bool_4 = (autos["unrepaired_damage"] == row)
    damage_price_list = autos[bool_4]["price"].mean().round()
    damage_price[row] = damage_price_list

#  establish a dataframe for analysis
dp_series = pd.Series(damage_price)
dp = pd.DataFrame(dp_series, columns=["mean price"])

print(dp)
print('----------------------')
print('price difference between damaged and undamaged:',dp["mean price"][0]-dp["mean price"][1])

     mean price
no       7164.0
yes      2241.0
----------------------
price difference between damaged and undamaged: 4923.0


Obviously, the car without damage can be sold at better price.

Based on column above, average price of the damaged car is approximately $4900 cheaper than that of the undamaged car.